In [1]:
import os

In [2]:
%pwd

'd:\\Coding\\Machine_Learning\\Kidney-Disease-Classificatioin-DeepLearning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Coding\\Machine_Learning\\Kidney-Disease-Classificatioin-DeepLearning'

In [5]:
import dagshub
dagshub.init(repo_owner='hozmha', repo_name='Kidney-Disease-Classification-DeepLearning', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as hozmha

Initialized MLflow to track repo "hozmha/Kidney-Disease-Classification-DeepLearning"

Repository hozmha/Kidney-Disease-Classification-DeepLearning initialized!

🏃 View run abrasive-ray-255 at: https://dagshub.com/hozmha/Kidney-Disease-Classification-DeepLearning.mlflow/#/experiments/0/runs/e9bcb4558b004d9a9adf441d36d239b0
🧪 View experiment at: https://dagshub.com/hozmha/Kidney-Disease-Classification-DeepLearning.mlflow/#/experiments/0


In [6]:
import tensorflow as tf

d:\miniconda\envs\kidney\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [7]:
model = tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    mlflow_uri: str
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath  = CONFIG_FILE_PATH,
        params_filepath  = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories(self.config.artifacts_root)
    

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/trained_model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri= "https://dagshub.com/hozmha/Kidney-Disease-Classification-DeepLearning.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE   
        )
        return eval_config


In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        degenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**degenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            class_mode = "sparse",
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
        

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 1.0000 - loss: 0.0326


2026/01/12 19:22:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/12 19:22:35 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2026/01/12 19:24:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run rare-roo-239 at: https://dagshub.com/hozmha/Kidney-Disease-Classification-DeepLearning.mlflow/#/experiments/0/runs/81ae87e5a81848eb9c2740d938408893
🧪 View experiment at: https://dagshub.com/hozmha/Kidney-Disease-Classification-DeepLearning.mlflow/#/experiments/0
